In [1]:
import numpy as np
import matplotlib.pyplot as plt
import receiving_operating_characteristic as roc
from sklearn import metrics
import collections

def intersect(a, b):
    return list(set(a) & set(b))

INFO_FIELDS = ['locus', 'rsid', 'stat', 'ld', 'causality']
class LDResult(collections.namedtuple('_LDResult', INFO_FIELDS)):
    __slots__ = ()

nloci = 20
nsim = 5

In [2]:
ldmatrix = [[] for x in range(nloci)]
lddir = '/mnt/storage/saikat/work/fine-mapping-SNPs/artificial_phenotype/metaprod/ldmap_nold/combined/'
for locus in range(nloci):
    rsidfile = lddir + 'Locus.%03i.LD.rsid' % (locus + 1)
    ldfile = lddir + 'Locus.%03i.LD' % (locus + 1)
    rsidlist = list()
    ldcol = collections.defaultdict(lambda:0)
    with open(rsidfile, 'r') as mfile:
        for mline in mfile:
            rsidlist.append(mline.split()[0])
    this_ldmat = np.loadtxt(ldfile)
    nsnps = len(rsidlist)
    for snp1 in range(nsnps):
        for snp2 in range(nsnps):
            rsid1 = rsidlist[snp1]
            rsid2 = rsidlist[snp2]
            ldcol[rsid1, rsid2] = np.square(this_ldmat[snp1, snp2])
    ldmatrix[locus] = ldcol

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/storage/saikat/work/fine-mapping-SNPs/artificial_phenotype/metaprod/ldmap_nold/combined/Locus.001.LD.rsid'

In [3]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

basedir='/mnt/storage/saikat/work/fine-mapping-SNPs/artificial_phenotype/metaprod/simulations_nold_feat/'

bammgwas = list()
bammgwas_feat = list()
caviarbf_c1 = list()
caviarbf_c2 = list()
paintor_comb = list()
paintor_indv = list()
paintor_fa = list()
finemap = list()
snptest = list()
bimbam = list()

for sim in range(50, nsim + 50):
    simdir = basedir + 'sim%03i/' % (sim + 1)

    causal_file = simdir + 'samples/causal.snplist'

    causal_rsids = [[] for x in range(200)]
    counter = 0
    with open(causal_file, 'r') as mfile:
        for mline in mfile:
            if not mline.startswith('#'):
                mline_split =  mline.split()
                if mline_split[0] == 'Locus':
                    counter += 1
                else:
                    causal_rsids[counter - 1].append(mline_split[0])
    causal_loci = [min(len(x), 1) for x in causal_rsids]

    # BammGWAS
    thisres = list()
    for locus in range(nloci):
        outfile = simdir + 'bvslr/zmax2_muvar_pi5.5_sig7.5/bvslr_meta_res/Locus.%03i.res' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        with open(outfile, 'r') as mfile:
            for mline in mfile:
                mline_split = mline.split()
                if not mline_split[0] == 'Causal':
                    rsid = mline_split[0]
                    prob = float(mline_split[4])
                    causality = 1 if rsid in causals else 0
                    ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                    mres = LDResult(locus = locus + 1,
                                    rsid = rsid,
                                    stat = prob,
                                    ld = ld,
                                    causality = causality)
                    thisres.append(mres)
    bammgwas.append(thisres)
    
    # BammGWAS with functional annotations
    thisres = list()
    for locus in range(nloci):
        outfile = simdir + 'bvslr/zmax2_muvar_pi5.5_sig7.5_feature/bvslr_meta_res/Locus.%03i.res' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        with open(outfile, 'r') as mfile:
            for mline in mfile:
                mline_split = mline.split()
                if not mline_split[0] == 'Causal':
                    rsid = mline_split[0]
                    prob = float(mline_split[4])
                    causality = 1 if rsid in causals else 0
                    ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                    mres = LDResult(locus = locus + 1,
                                    rsid = rsid,
                                    stat = prob,
                                    ld = ld,
                                    causality = causality)
                    thisres.append(mres)
    bammgwas_feat.append(thisres)

    # CAVIARBF c1
    thisres = list()
    for locus in range(nloci):
        rsidfile = simdir + 'caviarbf/c1_weighted/Locus.%03i' % (locus + 1)
        resfile = simdir + 'caviarbf/c1_weighted/Locus.%03i.prior0.marginal' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        rsidlist = list()
        with open(rsidfile, 'r') as mfile:
            for mline in mfile:
                rsidlist.append(mline.split()[0])
        with open(resfile, 'r') as mfile:
            for mline in mfile:
                mlinesplit = mline.split()
                index = int(mlinesplit[0]) - 1
                rsid = rsidlist[index]
                prob = float(mlinesplit[1])
                causality = 1 if rsid in causals else 0
                ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                mres = LDResult(locus = locus + 1,
                                rsid = rsid,
                                stat = prob,
                                ld = ld,
                                causality = causality)
                thisres.append(mres)
    caviarbf_c1.append(thisres)
    
    # CAVIARBF c2
    thisres = list()
    for locus in range(nloci):
        rsidfile = simdir + 'caviarbf/c2_weighted/Locus.%03i' % (locus + 1)
        resfile = simdir + 'caviarbf/c2_weighted/Locus.%03i.prior0.marginal' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        rsidlist = list()
        with open(rsidfile, 'r') as mfile:
            for mline in mfile:
                rsidlist.append(mline.split()[0])
        with open(resfile, 'r') as mfile:
            for mline in mfile:
                mlinesplit = mline.split()
                index = int(mlinesplit[0]) - 1
                rsid = rsidlist[index]
                prob = float(mlinesplit[1])
                causality = 1 if rsid in causals else 0
                ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                mres = LDResult(locus = locus + 1,
                                rsid = rsid,
                                stat = prob,
                                ld = ld,
                                causality = causality)
                thisres.append(mres)
    caviarbf_c2.append(thisres)

    # PAINTOR combined (= weighted) LD
    thisres = list()
    for locus in range(nloci):
        resfile = simdir + 'paintor/combined_LD/output/Locus.%03i.results' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        with open(resfile, 'r') as mfile:
            next(mfile)
            for mline in mfile:
                mlinesplit = mline.split()
                rsid = mlinesplit[2]
                prob = float(mlinesplit[4])
                causality = 1 if rsid in causals else 0
                ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                mres = LDResult(locus = locus + 1,
                                rsid = rsid,
                                stat = prob,
                                ld = ld,
                                causality = causality)
                thisres.append(mres)
    paintor_comb.append(thisres)
    
    # PAINTOR individual LD
    thisres = list()
    for locus in range(nloci):
        resfile = simdir + 'paintor/individual_LD/output/Locus.%03i.results' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        with open(resfile, 'r') as mfile:
            next(mfile)
            for mline in mfile:
                mlinesplit = mline.split()
                rsid = mlinesplit[1]
                prob = float(mlinesplit[7])
                causality = 1 if rsid in causals else 0
                ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                mres = LDResult(locus = locus + 1,
                                rsid = rsid,
                                stat = prob,
                                ld = ld,
                                causality = causality)
                thisres.append(mres)
    paintor_indv.append(thisres)
    
    # PAINTOR combined (= weighted) LD + functional annotations
    thisres = list()
    for locus in range(nloci):
        resfile = simdir + 'paintor/combined_LD_FA/output_fa/Locus.%03i.results' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        with open(resfile, 'r') as mfile:
            next(mfile)
            for mline in mfile:
                mlinesplit = mline.split()
                rsid = mlinesplit[2]
                prob = float(mlinesplit[4])
                causality = 1 if rsid in causals else 0
                ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                mres = LDResult(locus = locus + 1,
                                rsid = rsid,
                                stat = prob,
                                ld = ld,
                                causality = causality)
                thisres.append(mres)
    paintor_fa.append(thisres)
    
    # FINEMAP weighted LD
    thisres = list()
    for locus in range(nloci):
        resfile = simdir + 'finemap/c2_weighted/Locus.%03i.snp' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        with open(resfile, 'r') as mfile:
            next(mfile)
            for mline in mfile:
                mlinesplit = mline.split()
                rsid = mlinesplit[1]
                prob = float(mlinesplit[3])
                causality = 1 if rsid in causals else 0
                ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                mres = LDResult(locus = locus + 1,
                                rsid = rsid,
                                stat = prob,
                                ld = ld,
                                causality = causality)
                thisres.append(mres)
    finemap.append(thisres)   
    
    # PVALUES
    thisres = list()
    for locus in range(nloci):
        resfile = simdir + 'snptest/meta/Locus.%03i.meta.out' % (locus + 1)
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        with open(resfile, 'r') as mfile:
            for mline in mfile:
                if not mline.startswith('#') and not mline.startswith('chr'):
                    mlinesplit = mline.split()
                    rsid = mlinesplit[1]
                    pval = float(mlinesplit[5])
                    causality = 1 if rsid in causals else 0
                    ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                    mres = LDResult(locus = locus + 1,
                                    rsid = rsid,
                                    stat = -np.log(pval),
                                    ld = ld,
                                    causality = causality)
                    thisres.append(mres)                    
    snptest.append(thisres)


    # BIMBAM
    thisres = list()
    for locus in range(nloci):
        resfile = simdir + 'bimbam/meta/output/Locus.%03i.meta.ssd-bf.txt' % (locus + 1)
        nsnps = file_len(resfile) - 1
        causals = causal_rsids[locus]
        ldrsq = ldmatrix[locus]
        with open(resfile, 'r') as mfile:
            next(mfile)
            for mline in mfile:
                mlinesplit = mline.split()
                rsid = mlinesplit[0]
                log10bf = float(mlinesplit[1])
                pi = 2 / nsnps
                postodd = (np.power(10, log10bf) * pi) / (1 - pi)
                ppa = postodd / (1 + postodd)
                causality = 1 if rsid in causals else 0
                ld = max([ldrsq[rsid, x] for x in causals]) if len(causals) > 0 else 0
                mres = LDResult(locus = locus + 1,
                                rsid = rsid,
                                stat = ppa,
                                ld = ld,
                                causality = causality)
                thisres.append(mres)                   
    bimbam.append(thisres)

In [4]:
def precisionld_recall_curve(ldresult, ldcut):
    #epsilon = -100
    nitems = len(ldresult)
    estimated_values_rounded = np.zeros(nitems)
    for i in range(nitems):
    #    if ldresult[i].stat >= epsilon:
        estimated_values_rounded[i] = ldresult[i].stat
    sorted_index = np.argsort(estimated_values_rounded)[::-1]
    
    real_values = np.array([x.causality for x in ldresult])
    positives = np.sum(real_values)
    negatives = nitems - positives
    
    fpr = []
    tpr = []
    precision = []
    recall = []
    nselected = []
    false_positives = 0
    true_positives  = 0
    alpha = 0
    precision_ld = []
    true_positives_with_ld = 0
    tpld_prev = 0
    fdr = []
    j = 0
    while j < nitems:
        if not estimated_values_rounded[sorted_index[j]] == alpha:
            fpr.append(false_positives / negatives)
            tpr.append(true_positives / positives)
            if (true_positives + false_positives) == 0:
                precision.append(1.0)
                recall.append(0.0)
                precision_ld.append(1.0)               
            else:
                precision.append(true_positives / (true_positives + false_positives))
                recall.append(true_positives / positives)
                precision_ld.append(true_positives_with_ld / (true_positives + false_positives))
                
            if (false_positives == 0):
                fdr.append(0.0)
            else:
                fdr.append((true_positives_with_ld - true_positives) / false_positives)
            nselected.append(j)
            alpha = estimated_values_rounded[sorted_index[j]]

        if real_values[sorted_index[j]] == 1:
            true_positives += 1
        else:
            false_positives += 1
            
        if ldresult[sorted_index[j]].ld > ldcut:
            true_positives_with_ld += 1

        j += 1

    fpr.append(false_positives / negatives)
    tpr.append(true_positives / positives)
    precision.append(true_positives / (true_positives + false_positives))
    recall.append(true_positives / positives)
    precision_ld.append(true_positives_with_ld / (true_positives + false_positives))
    fdr.append((true_positives_with_ld - true_positives) / false_positives)
    nselected.append(nitems)

    return fpr, tpr, precision, recall, nselected, precision_ld, fdr

In [5]:
def xyplotvals(data, ldcut, nloci):
    #xvals = np.linspace(0, 200, 200)
    xvals = np.linspace(0, 1, 200)
    yests = list()
    for sim in range(nsim):
    #for sim in range(5):
        _fpr, _tpr, _precision, _recall, _nselected, _precisionld, _fdr = precisionld_recall_curve(data[sim], ldcut)
        #_nselected = np.array(_nselected) / nloci
        #yest = np.interp(xvals, _nselected, _recall)
        #yest = np.interp(xvals, _recall[1:], _precisionld[1:])
        #yest = np.interp(xvals, _recall[1:], _precision[1:])
        yest = np.interp(xvals, _recall[1:], _fdr[1:])
        yests.append(yest)
    yests = np.array(yests)
    yvals = yests.mean(axis=0)
    ystd = yests.std(axis=0)
    xvals = np.insert(xvals, 0, 0)
    yvals = np.insert(yvals, 0, 0)
    ystd = np.insert(ystd, 0, 0)
    return (xvals, yvals, ystd)

ldcut = 0.9

xybammgwas = xyplotvals(bammgwas, ldcut, nloci)
xybammgwas_feat = xyplotvals(bammgwas_feat, ldcut, nloci)
#xycaviarbf_c1 = xyplotvals(caviarbf_c1, ldcut, nloci)
xycaviarbf_c2 = xyplotvals(caviarbf_c2, ldcut, nloci)
xypaintor_comb = xyplotvals(paintor_comb, ldcut, nloci)
#xypaintor_indv = xyplotvals(paintor_indv, ldcut, nloci)
xypaintor_fa = xyplotvals(paintor_fa, ldcut, nloci)
xyfinemap = xyplotvals(finemap, ldcut, nloci)
xysnptest = xyplotvals(snptest, ldcut, nloci)
xybimbam = xyplotvals(bimbam, ldcut, nloci)

In [22]:
def plot_benchmark(ax, data, color, myls, legend):
    xvals = data[0]
    #xvals = np.insert(data[0], 0, 0)
    yvals = data[1]
    #yvals = np.insert(data[1], 0, 1)
    ystd = data[2]
    #ystd = np.insert(data[2], 0, 0)
    #yvals[0] = 0
    yupper = np.minimum(yvals + ystd, 1)
    ylower = yvals - ystd
    color = color
    linestyle = myls
    mlabel_roc_py = legend
    #ax.fill_between(xvals, ylower, yupper, color=color, alpha=0.15)
    coef = 4
    _dash = 1
    _dot = 0.3
    _dashspace = 0.8
    _dotspace = 0.5
    if myls == 'solid':
        mydash = []
    elif myls == 'dashed':
        mydash = [_dash, _dashspace]
    elif myls == 'dotted':
        mydash = [_dot, _dotspace]
    elif myls == 'dashdot':
        mydash = [_dash, _dashspace, _dot, _dashspace]
    elif myls == 'dashdotdot':
        mydash = [_dash, _dashspace, _dot, _dashspace, _dot, _dashspace]
    elif myls == 'dashdashdot':
        mydash = [_dash, _dashspace, _dashspace, _dashspace, _dot, _dashspace]
    mydash = [x * coef for x in mydash]
    ax.plot(xvals, yvals, color=color, dashes = mydash, lw=4, label=mlabel_roc_py)
    return

kelly_colors_hex = [
    '#FFB300', # Vivid Yellow
    '#803E75', # Strong Purple
    '#FF6800', # Vivid Orange
    '#A6BDD7', # Very Light Blue
    '#C10020', # Vivid Red
    '#CEA262', # Grayish Yellow
    '#817066', # Medium Gray

    # The following don't work well for people with defective color vision
    '#007D34', # Vivid Green
    '#F6768E', # Strong Purplish Pink
    '#00538A', # Strong Blue
    '#FF7A5C', # Strong Yellowish Pink
    '#53377A', # Strong Violet
    '#FF8E00', # Vivid Orange Yellow
    '#B32851', # Strong Purplish Red
    '#F4C800', # Vivid Greenish Yellow
    '#7F180D', # Strong Reddish Brown
    '#93AA00', # Vivid Yellowish Green
    '#593315', # Deep Yellowish Brown
    '#F13A13', # Vivid Reddish Orange
    '#232C16', # Dark Olive Green
    ]

marker_list = ['8', '>', 'd', '<', '*', 'p', '^', 's', 'h', 'v', 'D', r'$\clubsuit$']

bordercolor = '#2B2B2B'
bordercolor = '#333333'
borderwidth = 2
colors = kelly_colors_hex
figsize = (12,12)
axis_font_size = 30
label_font_size = 25
legend_font_size = 25

fig = plt.figure(figsize = figsize)
ax1 = fig.add_subplot(111)

#xx = [0, 200]
#yy = [0,1]
#ax1.plot(xx, yy, '--', color='dimgrey', lw=2)

plot_benchmark(ax1, xypaintor_comb, colors[3], 'dashdotdot', 'PAINTOR')
plot_benchmark(ax1, xypaintor_fa, colors[6], 'solid', 'PAINTOR-FA')
plot_benchmark(ax1, xycaviarbf_c2, colors[9], 'solid', 'CAVIARBF')
plot_benchmark(ax1, xyfinemap, colors[7], 'dashdashdot', 'FINEMAP')
plot_benchmark(ax1, xysnptest, colors[0], 'solid', 'SNPTEST / META')
plot_benchmark(ax1, xybimbam, colors[1], 'dashed', 'BIMBAM')
plot_benchmark(ax1, xybammgwas, colors[12], 'dashdotdot', 'BammGWAS')
plot_benchmark(ax1, xybammgwas_feat, colors[4], 'solid', 'BammGWAS-FA')


mxlabel = r'Recall'
#mxlabel = r'Average number of SNPs selected per locus'
mylabel = r'Fraction of false positives with $r_{\mathrm{c}}^2$ > 0.9'
#mylabel = r'Precision'
#mylabel = r'Recall'
ax1.set_xlabel(mxlabel, {'size': axis_font_size, 'color': bordercolor}, labelpad = 15)
ax1.set_ylabel(mylabel, {'size': axis_font_size, 'color': bordercolor}, labelpad = 20)

ax1.set_xlim(-0.05, 1.05)
#ax1.set_xlim(0, 200)
ax1.set_ylim(-0.05, 1.05)

#ax.axhline(y = -np.log10(5e-8), linestyle = 'dotted', color='dimgray')
#legendtitle = 'CardiogramPlusC4D'
#legend = ax.legend(loc='upper left', bbox_to_anchor=(0.05, 0.95),
#                   scatterpoints = 1,
#                   frameon = True, borderpad = 1.5, labelspacing = 1.5, title = legendtitle)
(lines, labels) = ax1.get_legend_handles_labels()
#it's safer to use linestyle='none' and marker='none' that setting the color to white
#should be invisible whatever is the background
#lines.insert(3, plt.Line2D([0, 0], [0, 0], linestyle='none'))
#labels.insert(3,'')

legend = ax1.legend(lines, labels,
                    loc='lower center', 
                    bbox_to_anchor=(0.5, 0.05),
                    handlelength = 6.4,
                    frameon = True,
                    borderpad = 1.5,
                    labelspacing = 1.5,
                    ncol = 1)
lframe = legend.get_frame()
lframe.set_edgecolor(bordercolor)
lframe.set_linewidth(borderwidth)
lframe.set_facecolor('white')

for fonts in ([legend.get_title()] + legend.texts):
    fonts.set_fontsize(legend_font_size)
    fonts.set_color(bordercolor)
ax1.tick_params(axis='both', which = 'major', 
               length = 10, width = borderwidth, pad=10,
               labelsize = label_font_size,
               color = bordercolor,
               labelcolor = bordercolor,
               bottom = True, top = False, left = True, right = False
              )
for side, border in ax1.spines.items():
    border.set_linewidth(borderwidth)
    border.set_color(bordercolor)
ax1.grid(color='dimgray', lw=0.5, alpha=0.5)

plt.tight_layout()
#plt.savefig('simulation_finemap_benchmark.pdf', bbox_inches='tight')
#plt.savefig('simu_finemap_recall_nold.pdf', bbox_inches='tight')
#plt.savefig('simu_finemap_altprecision_recall_nold.pdf', bbox_inches='tight')
#plt.savefig('simu_finemap_precision_recall_nold.pdf', bbox_inches='tight')
#plt.savefig('simu_finemap_recall_withld.pdf', bbox_inches='tight')
plt.show()